In [1]:
#Ensure we are running Python 3
import sys
pyinfo = sys.version_info
assert (pyinfo.major == 3), "Python 3 required. The version you are using is: {}".format(pyinfo) 

In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import pickle
%matplotlib inline

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
#Perspective warping...These values work for the test image sequences
#Warping from the ROI to Bird Eye View. Creating a small image coz it saves time. large image probably
#doesn't give any thing extra
p1 = [402, 10]#for full ROI warp 
p2 = [530, 10]#for full ROI warp
#p1 = [350, 30] #for a little lesser...cleaner, but shorter data
#p2 = [540, 30] #for a little lesser...cleaner, but shorter data
p3= [930, 270]
p4= [15, 270]

src_pts = np.array([p1, p2, p3, p4], dtype=np.float32)
offset = 100
dst_pts = np.array([[offset, 0], [640-offset, 0], [640-offset,240], [offset, 240]], dtype=np.float32)
M = cv2.getPerspectiveTransform(src_pts, dst_pts)


def warp(img, M):
    out = cv2.warpPerspective(img, M, (640, 480), flags=cv2.INTER_LINEAR)
    return out

In [ ]:
def get_calibration(calib_name):
    calibration = pickle.load(open(calib_name, "rb"))
    return calibration
camera_calibration_dir = os.path.join(os.getcwd(),"camera_cal")
calib_name = os.path.join(camera_calibration_dir,"calibration_pickle.p")
calibration = get_calibration(calib_name)
print(calibration)
mtx = calibration['mtx']
dist = calibration['dist']

In [ ]:
"""
Run the processing pipeline on image
if return_at == -1, all steps are perfomed, else, steps are performed till the required number
"""
from AdvancedLaneDetection.utilities import *
import AdvancedLaneDetection.BinaryThreshold as bt

R_Range = bt.Threshold(140, 250, 5)
V_Range = bt.Threshold(140, 240, 5)

def processImage(image, return_at = -1):
    
    Y,X,C = image.shape
    lines = None
    
    #undistort image
    image = cv2.undistort(image, mtx, dist, None, mtx)
    
    #extract image ROI
    #Most parts of the image do not provide useful info
    #roi = drawROI(image, *(200, 400), *(1080, 720)) 
    ROI_x1, ROI_y1, ROI_x2, ROI_y2 = 175, 450, 1105, 720 
    roi = get_ROI(image, ROI_x1, ROI_y1, ROI_x2, ROI_y2)
    plt.imshow(roi)
    #plt.show()
    
    bin_img,sv,r = bt.binary_threshold(roi, R_Range, V_Range)
    #plt.imshow(bin_img)
    #plt.show()
    #weighted_image = cv2.addWeighted(bin_img, 0.8, roi, 0.2, 0)
    p1_ = [594,450]
    p2_ = [687,450]
    p3_ = [1118,720]
    p4_ = [193,720]
    out_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    #polygon = drawPolygon(out_image, p1_, p2_, p3_, p4_)
    #out_image[ROI_y1:ROI_y2, ROI_x1:ROI_x2, :] = bin_img
    #bin_img = drawPolygon(bin_img, p1, p2, p3, p4)
    
    warped_image = warpImage(bin_img, M, (640, 240))
    out_image[0:240, -640:] = warped_image
    return lines, out_image

In [ ]:
img_dir = "test_images"
for img_name in os.listdir(img_dir):
    #ONly look at image files:
    if (img_name):# ==  "test5.jpg"):
        if os.path.isfile(os.path.join("test_images", img_name)):
            print(img_name)
            img_in_path = os.path.join("test_images", img_name);

            image = mpimg.imread(img_in_path)
            lines, out_image = processImage(image, -1)
            #mpimg.imsave(img_out_path, out_image)
            img_name = os.path.splitext(os.path.basename(img_name))[0]
            img_out_path = os.path.join("test_images", "output", img_name);
            cv2.imwrite(img_out_path+".png", out_image)
        
print("Done")

In [ ]:
import imageio.plugins
import moviepy.editor as mp

In [ ]:
# Import everything needed to edit/save/watch video clips
import imageio.plugins
import moviepy.editor as mp
from moviepy.editor import VideoFileClip
from IPython.display import HTML



In [ ]:
def process_image(image, return_at=-1):#(return_at, image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    _, result = processImage(image, return_at)
    return result

In [ ]:
white_output = 'project_out.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

In [ ]:
yellow_output = 'challenge_out.mp4'
clip2 = VideoFileClip('challenge_video.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
yellow_output = 'harder_out.mp4'
clip2 = VideoFileClip('harder_challenge_video.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
a = 255 * np.ones((10,10, 3), dtype=np.uint8)
a[5:10, 3:10, :] = 0
b = 255 * np.ones((10,10, 3), dtype=np.uint8)

In [ ]:
c = cv2.bitwise_and(a, b)

In [ ]:
d_c= cv2.cvtColor(c, cv2.COLOR_BGR2RGB)
plt.imshow(d_c, cmap="gray")

In [ ]:
c

In [ ]:
c.shape

In [ ]:
import cv2
vidcap = cv2.VideoCapture('project_video.mp4')
success,image = vidcap.read()

success,image = vidcap.read()
print(success)
count = 0
success = True
while success and count < 5:
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file
  count += 1

In [ ]:
clip1 = VideoFileClip("project_video.mp4")

In [ ]:
vidcap.get()